In [1]:
from IPython.display import clear_output

In [2]:
!pip install pymorphy2 sparse_dot_topn lightfm
!python -m nltk.downloader stopwords
!python -m nltk.downloader wordnet

clear_output()

In [3]:
import os
import re

import json
import gzip
import codecs

from itertools import islice, chain, filterfalse
from collections import Counter, defaultdict
from operator import itemgetter

import numpy as np
import scipy.sparse as sp
import pandas as pd

import joblib

import lxml.html as lhtml

# from tqdm.notebook import tqdm
from tqdm import tqdm

In [4]:
WORKDIR = '.'

In [5]:
!mkdir -p "{WORKDIR}/data" "{WORKDIR}/models"

In [6]:
def save_array(a, filename: str, sparse: bool = False, **params):
    if sparse and not sp.issparse(a):
        a = sp.csr_matrix(a)
    elif not sparse and sp.issparse(a):
        a = np.asarray(a.todense())

    with open(filename, 'wb') as f_data:
        save = sp.save_npz if sparse else np.save
        return save(f_data, a, **params)


def load_array(filename: str, sparse: bool = False, **params):
    with open(filename, 'rb') as f_data:
        load = sp.load_npz if sparse else np.load
        return load(f_data, **params)

In [7]:
def parse_specializations(s):
    res = s[1:-1].split(',')
    res = map(int, res)
    res = list(res)
    # res = np.asarray(res, dtype=int)
    return res

vacancies_file = os.path.join(WORKDIR, 'data/vacancies_info.csv.gz')

if not os.path.isfile(vacancies_file):
    # Загружаем специализации для обучения
    df_train_ids = pd.read_csv(
        os.path.join(WORKDIR, 'train_labels.csv.gz'),
        index_col='vacancy_id',
        compression='gzip',
    )

    df_train_ids['specializations'] = df_train_ids['specializations'].map(parse_specializations)
    df_train_ids['is_train'] = True

    # Загружаем специализации для теста
    df_test_ids = pd.read_csv(
        os.path.join(WORKDIR, 'test_vacancy_ids.csv.gz'),
        index_col='vacancy_id',
        compression='gzip',
    )

    # Объединяем в один датафрейм
    df_all_ids = pd.concat([df_train_ids, df_test_ids], axis=0)
    df_all_ids['is_train'].fillna(False, inplace=True)
    df_all_ids.sort_index(inplace=True)

    # Загружаем информацию о каждой из вакансий
    df_vacancies_info = pd.read_csv(
        os.path.join(WORKDIR, 'vacancies_info.csv.gz'),
        index_col='vacancy_id',
        compression='gzip',
    )

    # Объединяем в один датафрейм
    df_all_ids = pd.merge(df_all_ids, df_vacancies_info, left_index=True, right_index=True, how='left')

    df_all_ids.to_csv(vacancies_file, index=True, compression='gzip')
else:
    df_all_ids = pd.read_csv(
        vacancies_file,
        index_col='vacancy_id',
        compression='gzip',
    )
    df_all_ids.loc[df_all_ids['is_train'], 'specializations'] = \
        df_all_ids.loc[df_all_ids['is_train'], 'specializations'].map(parse_specializations)

df_all_ids.head()

/home/poisk/.conda/envs/ml-py37/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,specializations,is_train,area_id,compensation_from,compensation_to,creation_date,currency,employer,employment,work_experience,work_schedule
vacancy_id,,,,,,,,,,,
1,"[242, 256, 302, 324, 358, 440]",True,26,22000.0,24000.0,2019-01-24,RUR,0ce23382345c,full,between1And3,fullDay
2,NaN,False,160,NaN,NaN,2019-07-26,NaN,b9aa259f8724,full,between1And3,fullDay
3,[211],True,1002,NaN,NaN,2019-04-15,NaN,11ecc72a7a76,project,between1And3,fullDay
4,"[389, 412, 437]",True,22,NaN,36000.0,2019-07-12,RUR,e1e424ceb5e4,full,noExperience,fullDay
5,NaN,False,1002,600.0,NaN,2019-01-17,BYR,943fd4a3770a,full,between1And3,fullDay


In [8]:
df_all_ids['joined_work'] = (
    df_all_ids['employment'] + '_' +
    df_all_ids['work_experience'] + '_' +
    df_all_ids['work_schedule']
)

df_all_ids.head()

,specializations,is_train,area_id,compensation_from,compensation_to,creation_date,currency,employer,employment,work_experience,work_schedule,joined_work
vacancy_id,,,,,,,,,,,,
1,"[242, 256, 302, 324, 358, 440]",True,26,22000.0,24000.0,2019-01-24,RUR,0ce23382345c,full,between1And3,fullDay,full_between1And3_fullDay
2,NaN,False,160,NaN,NaN,2019-07-26,NaN,b9aa259f8724,full,between1And3,fullDay,full_between1And3_fullDay
3,[211],True,1002,NaN,NaN,2019-04-15,NaN,11ecc72a7a76,project,between1And3,fullDay,project_between1And3_fullDay
4,"[389, 412, 437]",True,22,NaN,36000.0,2019-07-12,RUR,e1e424ceb5e4,full,noExperience,fullDay,full_noExperience_fullDay
5,NaN,False,1002,600.0,NaN,2019-01-17,BYR,943fd4a3770a,full,between1And3,fullDay,full_between1And3_fullDay


In [9]:
def make_onehot_csr_matrix(s: pd.Series):
    mapping = defaultdict(lambda: len(mapping))

    data = np.ones(shape=(s.shape[0], ))
    indices = [mapping[k] for k in s]
    indptr = np.arange(0, len(data) + 1)

    X = sp.csr_matrix((data, indices, indptr), shape=(len(indptr) - 1, len(mapping)))
    mapping.default_factory = None
    mapping_inv = sorted(mapping, key=lambda e: mapping[e])

    return mapping, mapping_inv, X

In [10]:
mapping_employer, mapping_employer_inv, features_employer = \
    make_onehot_csr_matrix(df_all_ids['employer'])

features_employer.shape

(2912650, 345193)

In [11]:
def make_onehot_multiple_csr_matrix(s: pd.Series):
    mapping = defaultdict(lambda: len(mapping))

    data, indices, indptr = [], [], [0, ]

    for row in tqdm(s):
        if isinstance(row, list):
            row = list(map(lambda e: mapping[e], row))
        else:
            row = []

        data.extend([1] * len(row))
        indices.extend(row)
        indptr.append(len(data))

    X = sp.csr_matrix((data, indices, indptr), shape=(len(indptr) - 1, len(mapping)))
    mapping.default_factory = None
    mapping_inv = sorted(mapping, key=lambda e: mapping[e])

    return mapping, mapping_inv, X

In [12]:
mapping_spec, mapping_spec_inv, y_spec = \
    make_onehot_multiple_csr_matrix(df_all_ids.loc[:, 'specializations'])

y_spec.shape

100%|██████████| 2912650/2912650 [00:03<00:00, 841517.98it/s]


(2912650, 620)

In [13]:
vacancies_parts = (f for f in os.listdir(WORKDIR) if f.startswith('vacancies-'))
vacancies_parts = sorted(vacancies_parts)
vacancies_parts

['vacancies-01.json.gz',
 'vacancies-02.json.gz',
 'vacancies-03.json.gz',
 'vacancies-04.json.gz',
 'vacancies-05.json.gz',
 'vacancies-06.json.gz',
 'vacancies-07.json.gz',
 'vacancies-08.json.gz',
 'vacancies-09.json.gz',
 'vacancies-10.json.gz']

In [14]:
def read_vacancies_part(filename):
    with gzip.open(filename, mode='r') as f_gz:
        records = json.load(f_gz)
        records = {int(k): v for k, v in records.items()}
    return records

In [15]:
from functools import lru_cache

from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer


ru_morph = MorphAnalyzer()

@lru_cache(maxsize=15000)
def morph_process(token):
    return ru_morph.parse(token)[0].normal_form


stop_words = map(morph_process, stopwords.words('russian'))
stop_words = stopwords.words('russian') + list(stop_words)

In [16]:
vacancies_it = map(lambda p: os.path.join(WORKDIR, p), tqdm(vacancies_parts))
vacancies_it = map(read_vacancies_part, vacancies_it)
vacancies_it = ((k, v) for p in vacancies_it for k, v in p.items())

index = []

def content_names_reader(index):
    for vacancy_id, vacancy_info in vacancies_it:
        name = re.sub('\(.*?\)', '', vacancy_info['name'].lower())
        index.append(vacancy_id)
        yield name

content_names = content_names_reader(index)
content_names = tqdm(content_names, position=0)

0it [00:00, ?it/s]/10 [00:00<?, ?it/s]

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

def create_tfidf_vectorizer(**params):
    vec = TfidfVectorizer(
        stop_words=stop_words,
        preprocessor=morph_process,
        ngram_range=(1, 2),
        min_df=5,
        **params
    )
    return vec

In [18]:
content_names_array_file = os.path.join(WORKDIR, 'data/content_names_2.npz')
content_names_terms_idfs = os.path.join(WORKDIR, 'data/content_names_2.idf')
content_names_vacancies_mapping = os.path.join(WORKDIR, 'data/content_names_2.mapping')

if not os.path.isfile(content_names_array_file):
    vec = create_tfidf_vectorizer()

    # Считаем tfidf-вектора и сохраняем их
    features_content_names = vec.fit_transform(content_names)
    save_array(features_content_names, content_names_array_file, sparse=True)

    # Сохраняем словарик с idf
    vocabulary_inv = sorted(vec.vocabulary_, key=lambda e: vec.vocabulary_[e])
    with open(content_names_terms_idfs, mode='w', encoding='utf8') as f_data:
        for word, idf in zip(vocabulary_inv, vec.idf_):
            print(word, "%.16f" % idf, sep='\t', file=f_data)

    # Сохраняем порядок вакансий в матрице
    with open(content_names_vacancies_mapping, mode='w') as f_data:
        print(*index, sep='\n', file=f_data)
else:
    # Грузим tfidf-вектора
    features_content_names = load_array(content_names_array_file, sparse=True)

    # Грузим TfIdfVectorizer
    with open(content_names_terms_idfs, mode='r') as f_data:
        f_data = map(lambda s: s.rstrip().split('\t'), f_data)

        vocabulary_inv, vocabulary_idf = [], []
        for i, (word, idf) in enumerate(f_data):
            vocabulary_inv.append(word)
            vocabulary_idf.append(float(idf))

    vec = create_tfidf_vectorizer(vocabulary=vocabulary_inv)
    vec.idf_ = np.asarray(vocabulary_idf, dtype=float)

    # Грузим порядок документов
    with open(content_names_vacancies_mapping, mode='r') as f_data:
        index = list(map(int, f_data))

# Убеждаемся, что все правильно
assert (np.asarray(index) == df_all_ids.index.values).all()
assert features_content_names.shape == (df_all_ids.shape[0], len(vec.idf_))

features_content_names.shape

(2912650, 139435)

In [19]:
def get_best_ranks(ranks: np.ndarray, top: int, axis: int = 0, return_ranks: bool = False):
    top_slice = (slice(None), ) * axis + (slice(-top, None), )
    inv_slice = (slice(None), ) * axis + (slice(None, None, -1), )

    if top < ranks.shape[axis]:
        indices = np.argpartition(ranks, -top, axis=axis)[top_slice]
        ranks_top = np.take_along_axis(ranks, indices, axis=axis)
        indices = np.take_along_axis(indices, ranks_top.argsort(axis=axis)[inv_slice], axis=axis)
    else:
        indices = np.argsort(ranks, axis=axis)[top_slice]
        indices = indices[inv_slice]

    result = (indices, )

    if return_ranks:
        ranks = np.take_along_axis(ranks, indices, axis=axis)
        result += (ranks, )

    return result if len(result) > 1 else result[0]

In [20]:
def f1score(y_true, y_pred):
    # y_true = set(y_true)
    # y_pred = set(y_pred)
    
    tp = len(y_true & y_pred)
    precision = tp / len(y_pred)
    recall = tp / len(y_true)
    if precision == 0.0 and recall == 0.0:
        score = 0.0
    else:
        score = 2 * precision * recall / (precision + recall)
    return score

In [21]:
from sklearn.model_selection import train_test_split

mask = df_all_ids['is_train'].values.ravel()

indices_train, indices_valid = train_test_split(np.where(mask)[0], test_size=0.3, random_state=9872)
indices_test = np.where(~mask)[0]

y_spec_train = y_spec.copy().tolil()
y_spec_train[indices_valid] = 0
y_spec_train = y_spec_train.tocsr()

In [22]:
def lightfm_predict(model, item_features=None, user_features=None, item_indices=None, user_indices=None):
    user_bias, user_embedding = model.get_user_representations(features=user_features)
    item_bias, item_embedding = model.get_item_representations(features=item_features)
    
    if user_indices is not None:
        user_bias, user_embedding = user_bias[user_indices], user_embedding[user_indices]
    
    if item_indices is not None:
        item_bias, item_embedding = item_bias[item_indices], item_embedding[item_indices]
    
    y_pred = np.matmul(user_embedding, item_embedding.T) + item_bias[np.newaxis, :] + user_bias[:, np.newaxis]
    return y_pred

In [23]:
def make_predict(model, indices, top=3, batch_size=2048, return_ranks=False, verbose=True):
    y_pred, y_true, ranks_pred_all = [], [], []
    
    batches = range(0, indices.shape[0], batch_size)
    if verbose:
        batches = tqdm(batches, position=0, leave=False)
    
    for i in batches:
        j = min(i + batch_size, indices.shape[0])

        y_batch_pred = model.predict_batch(
            user_indices=indices[i:j],
        )
        
        res = get_best_ranks(y_batch_pred, top=top, axis=1, return_ranks=return_ranks)
        if return_ranks:
            indices_pred, ranks_pred = res
        else:
            indices_pred, ranks_pred = res, None
            
        y_pred.append(indices_pred)
        ranks_pred_all.append(ranks_pred)
            
    y_true = y_spec[indices]
    y_true = y_true if y_true.nnz else None
    y_pred = np.vstack(y_pred)
    
    res = (y_true, y_pred, )
    
    if return_ranks:
        ranks_pred = np.vstack(ranks_pred_all)
        res += (ranks_pred, )

    return res

In [24]:
def validate(model, indices, top=3):
    y_true, y_pred = make_predict(model, indices, top=top, return_ranks=False)
    
    y_true = list(map(set, y_true.tolil().rows))
    y_pred = list(map(set, y_pred))

    assert len(y_true) == len(y_pred)

    scores = tqdm(zip(y_true, y_pred), total=len(y_true), position=0, leave=False)
    scores = [f1score(*pair) for pair in scores]
    scores = np.asarray(scores)
    return np.mean(scores)

In [25]:
from lightfm import LightFM as BasicLightFM


class LightFM(BasicLightFM):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def eval(self, item_features=None, user_features=None):
        self._user_bias, self._user_embedding = self.get_user_representations(features=user_features)
        self._item_bias, self._item_embedding = self.get_item_representations(features=item_features)
        
    def predict_batch(self, item_indices=None, user_indices=None, use_item_bias=True, use_user_bias=True):
        user_bias, user_embedding = self._user_bias, self._user_embedding
        item_bias, item_embedding = self._item_bias, self._item_embedding
        
        if user_indices is not None:
            user_bias, user_embedding = user_bias[user_indices], user_embedding[user_indices]

        if item_indices is not None:
            item_bias, item_embedding = item_bias[item_indices], item_embedding[item_indices]

        y_pred = np.matmul(user_embedding, item_embedding.T)
        if use_item_bias:
            y_pred += item_bias[np.newaxis, :]
        if use_user_bias:
            y_pred += user_bias[:, np.newaxis]
        return y_pred

    
model = LightFM(
    no_components=200, 
    loss='warp', 
    learning_rate=0.03, 
    max_sampled=400, 
    random_state=1,
    user_alpha=1e-05,
)

In [26]:
%%time

score_best = 0

for epoch_i in range(10):
    model.fit_partial(
        y_spec_train,
        epochs=1,
        num_threads=32,
        user_features=features_content_names,
    )
    
    model.eval(user_features=features_content_names)
    score = validate(model, indices_valid, top=3)
    print('Epoch: {:02d}; f-measure = {:.6f}'.format(epoch_i + 1, score))
    
    if score > score_best:
        model_name = os.path.join(WORKDIR, 'models/lightfm_best.bin')
        joblib.dump(model, open(model_name, 'wb'))
        score_best = score

Epoch: 01; f-measure = 0.488451


Epoch: 02; f-measure = 0.496224


Epoch: 03; f-measure = 0.498689


Epoch: 04; f-measure = 0.500210


Epoch: 05; f-measure = 0.500713


Epoch: 06; f-measure = 0.500761


Epoch: 07; f-measure = 0.500418


Epoch: 08; f-measure = 0.500254


Epoch: 09; f-measure = 0.499656


Epoch: 10; f-measure = 0.499450
CPU times: user 2h 46min, sys: 41min 47s, total: 3h 27min 48s
Wall time: 10min 3s


In [27]:
model_name = os.path.join(WORKDIR, 'models/lightfm_best.bin')
model = joblib.load(open(model_name, 'rb'))

In [34]:
_, indices_valid, ranks_valid = make_predict(model, indices_valid, top=15, return_ranks=True)

In [35]:
ranks_valid.shape

(436898, 15)

In [36]:
indices_valid_file = os.path.join(WORKDIR, 'data/lightfm-valid.indices.npz')
ranks_valid_file = os.path.join(WORKDIR, 'data/lightfm-valid.ranks.npz')

save_array(ranks_valid, ranks_valid_file, sparse=False)
save_array(indices_valid, indices_valid_file, sparse=False)

In [37]:
_, indices_test, ranks_test = make_predict(model, indices_test, top=15, return_ranks=True)

In [38]:
ranks_test.shape

(1456325, 15)

In [42]:
indices_test_file = os.path.join(WORKDIR, 'data/lightfm-test.indices.npz')
ranks_test_file = os.path.join(WORKDIR, 'data/lightfm-test.ranks.npz')

save_array(ranks_test, ranks_test_file, sparse=False)
save_array(indices_test, indices_test_file, sparse=False)

In [41]:
%%time

def convert_specializations(s):
    s = set(s)
    s = map(lambda e: mapping_spec_inv[e], s)
    s = sorted(s)
    # s = np.asarray(s, dtype=int)
    return s

use_smart = False

y_pred_all = []

if use_smart:
    mask = ranks_test > threshold_best
    mask[:, 0] = True

    indices_pred = np.where(mask, indices_test, -1)
    indices_pred = map(lambda e: filter(lambda x: x >= 0, e), indices_pred)
else:
    indices_pred = indices_test[:,:3]

y_pred_all = list(map(convert_specializations, indices_pred))

df_submission = df_all_ids.loc[~df_all_ids['is_train'], ['specializations']]
df_submission['specializations'] = y_pred_all

submission_id = 21
submission_file = os.path.join(WORKDIR, 'submission_{:03d}.csv.gz').format(submission_id)

df_submission.to_csv(submission_file, index=True, compression='gzip')
df_submission.head()

CPU times: user 16.6 s, sys: 894 ms, total: 17.5 s
Wall time: 17.7 s


,specializations
vacancy_id,
2,"[82, 211, 278]"
5,"[494, 497, 541]"
7,"[495, 556, 588]"
8,"[83, 287, 387]"
10,"[205, 264, 429]"
